---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].
       'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
*This function should return a DataFrame with 20 columns and 15 entries.*

In [3]:
import pandas as pd
import numpy as np
import re


In [4]:


Location = "Energy Indicators.xls"
xfile = pd.ExcelFile(Location)
sheets = xfile.sheet_names
energy = xfile.parse(sheets[0], skiprows = np.arange(1,18), skip_footer = 38,  \
                     names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], parse_cols = [2,3,4,5])




#print(energy.iloc()[4]["Energy Supply"].str.rfind('...'))
#print(energy[energy["Country"] == "Guam"])

Crit1 = energy["Energy Supply"].str.find('..') == 0
energy.loc[Crit1,"Energy Supply"] = np.NaN
energy["Energy Supply"] = energy["Energy Supply"]*1000000.0

Crit2 = energy["Energy Supply per Capita"].str.find('..') == 0
energy.loc[Crit2,"Energy Supply per Capita"] = np.NaN
energy["Energy Supply per Capita"] = energy["Energy Supply per Capita"]*1.0

energy["Country"] = energy["Country"].str.replace(" +\(.*\)","") 
energy["Country"] = energy["Country"].str.replace("[0-9]+","")

ch_list = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}

for a in ch_list:
    energy.loc[energy["Country"] == a,"Country"] = ch_list[a] 

#energy[energy["Country"] == "Iran, Islamic Rep."] = "Iran" 



In [5]:
GPD = pd.read_csv("world_bank.csv", header = 4)
ch_list = {"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"}
for a in ch_list:   
    GPD.loc[GPD["Country Name"] == a, "Country Name"] = ch_list[a]


In [6]:

ScimEn = pd.read_excel("scimagojr-3.xlsx")
ScimEn_reduce = ScimEn[ScimEn["Rank"] <= 15]


In [16]:
df = pd.merge(ScimEn_reduce, energy, how = "inner",left_on = ["Country"], right_on = ["Country"] )
GPD2 = GPD[["Country Name","2006", '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
df = df.merge(GPD2, how = "inner", left_on = ["Country"], right_on = ["Country Name"])
df = df.set_index("Country")

del  df["Country Name"]
df["Energy Supply"] = pd.to_numeric(df["Energy Supply"])
df["Energy Supply per Capita"] = pd.to_numeric(df["Energy Supply per Capita"])
#df.shape

In [17]:
def answer_one():
    return df

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [18]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [247]:
def answer_two():
    df2 = pd.merge(ScimEn, energy, how = "inner",left_on = ["Country"], right_on = ["Country"] )
    df2 = df2.merge(GPD, how = "inner", left_on = ["Country"], right_on = ["Country Name"])
    return df2.shape[0]-14 #162
answer_two()

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [248]:
def answer_three():
    Top15 = answer_one()
    Top15 = Top15[["2006", '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    avgGPD = pd.Series()    
    for i in range(0,15):
        a = Top15.iloc()[i].dropna() 
        
        avgGPD.loc[a.name] = np.average(a)
    return avgGPD.sort_values('index',ascending = False)


### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [249]:
def answer_four():
    Top15 = answer_one()
    GPD_range = answer_three()
    index =GPD_range.index[5]
    line = Top15.loc[index]
    
    return line["2015"] - line["2006"]


### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [250]:
def answer_five():
    Top15 = answer_one()
    
    return Top15["Energy Supply per Capita"].mean()


### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [251]:
def answer_six():
    Top15 = answer_one()
    return (Top15["% Renewable"].idxmax(),Top15["% Renewable"].max())


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [252]:
def answer_seven():
    Top15 = answer_one()
  
    Top15["ratio"] = Top15["Self-citations"]/Top15["Citations"]
    return (Top15["ratio"].idxmax(),Top15["ratio"].max())


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [253]:
def answer_eight():
    Top15 = answer_one()   
    Top15["PopEst"] = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
   
    return Top15["PopEst"].sort_values(ascending = False).index[2]


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [254]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    cor = Top15[["Citable docs per Capita","Energy Supply per Capita"]].corr()
    return cor.iloc[0][1]

In [255]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [256]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [257]:
def answer_ten():
    Top15 = answer_one()
    median = Top15["% Renewable"].median()
    Top15["HighRenew"] = 0
    Crit = Top15["% Renewable"] >= median
    d = Top15[Crit].shape[0]   
    Top15.loc[Crit,"HighRenew"] = np.ones(d, dtype = "int")
    Top15["Rank"].sort_values()
    
    HR = pd.Series(Top15["HighRenew"])
   
    return  HR


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [21]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
  
    Top15["PopEst"] = Top15["Energy Supply"]/Top15["Energy Supply per Capita"]
    
    byconts = Top15["PopEst"].groupby(by = ContinentDict)
  
  
    s1 = byconts.size()
    s2 = byconts.sum()
    s3 = byconts.mean()
    s4 = byconts.std()
   
   
    
    DR = pd.DataFrame({"size":s1, "sum":s2, "mean":s3,"std":s4})
    del Top15["PopEst"]
    return DR


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [30]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15["Continents"] = pd.Series(ContinentDict)
    Top15["bins"] = pd.cut(Top15["% Renewable"],5)
    group = Top15.groupby(["Continents","bins"])
   
    return group.size()



### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [53]:
def answer_thirteen():
    Top15 = answer_one()
    Top15["PopEst"] = (Top15["Energy Supply"]/Top15["Energy Supply per Capita"])
    Top15["PopEst"] = Top15["PopEst"].map('{0:,f}'.format)
    return Top15["PopEst"]


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [40]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [42]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!

In [59]:
x = np.random.binomial(20,0.5,1000)
print(x)

[10 11  9 10  9 15  9 10 10 14  7  8  8  7 13 11  8 12  9  9  9  7 10 10  9
  8  5 11 15 10 10  9 13 12  6 10 10 13  8  9 10 13 10 10 16  7 11 12 11 11
  9 12  9  9 14 10 12  7  7 11 11 11 11 12  8  7 10 12 12 14  9 12 12 11 10
  9 11 10  9 10 10  9  8 10  8  9 11 11 13  7 10  9  9 13 10  7  9 12  9  9
  6 11  9 10 14 11  9 15 12  8 11 10 12  9  4  9 11 10 10  8  9 10 12 11 12
 12 12  6 11 14 11 11  9 11 13 11 12 10  7 10  7 14 15 10  7 13  8  8 12  7
 10  8 15  9  8  9  9  7  9  8 11 11 13  6  9 13 10 11  9  7 10  5 10 12  9
  9 10  5 11 12 11 11 13 14  9  6 14  8  5 11 11 13 12 13 11 12 12 10  7 11
  7  8 13 13 14 12 14 13  7  7  8  8 10 11  5 11 11 11 11 12 11 12 11 11  7
 10 11  5 11 10 15 12  9 11  9 10  7 11  7  8 11  9  9 12 10 12 13  8 11 10
 13 11 10  7  8 11 13 11  5  9  9 10  8 11 10 14 13 10  8 12  9 10 11  6  9
 10 12 13 15  8 12 10  9  9 13  8  8 11  8  9 11 11  8 11 10 10 10 11  6 12
  9 10 10  8 10  9 11  7  9 11  8 10  9 11 10  8  8  9 10 13 11 12  7  8 10
 11  5 11  7

In [61]:
print((x>=15).mean())

0.028
